# Leveraging Mathematical Modeling for Marketing Campaign Optimization
                                                    ~ Riya Kore & Tanisha Raaj 

## 1. Introduction ##

In today's competitive business landscape, optimizing marketing campaigns is essential for growth. The challenge is to allocate limited resources across various channels to maximize return on investment (ROI). This project aims to develop a mathematical model to optimize marketing budget distribution.

Traditionally, marketing decisions have relied on intuition rather than data-driven analysis. However, the rise of advanced computational tools and data availability has enabled the development of sophisticated models to enhance decision-making. Optimizing marketing efforts is vital in a dynamic environment where consumer preferences and behaviors are constantly changing. Effective budget allocation can significantly improve campaign performance and revenue, while poor allocation can lead to wasted resources and missed opportunities.

Marketing optimization has a rich history that dates back to the early days of advertising. Initially, marketers relied on simple heuristics and rules of thumb to allocate budgets. With the advent of digital advertising and the explosion of data, the need for more precise and efficient methods became evident. Today, mathematical models and algorithms play a crucial role in marketing strategy, providing businesses with the tools to make informed decisions and stay competitive in a crowded marketplace.

Our project will create a linear programming model to optimize marketing budgets, considering factors such as financial constraints, platform costs, audience reach, channel efficacy, and demographic or geographic requirements. The data driving our model will be sourced from historical performance metrics of major digital advertising platforms and past marketing initiatives, supplemented by synthetic data to fill any gaps.

Key data sources include the Social Media Advertising Dataset, Marketing Spending Analysis dataset, and Marketing Campaign Performance dataset from Kaggle. These datasets provide valuable insights into expenditure, reach, and ROI metrics, forming the foundation of our model. Historical data allows us to understand past trends and performance, while synthetic data helps simulate various scenarios and test the robustness of our model.

Our model will aim to maximize ROI while adhering to budgetary and operational constraints. It will allocate budgets across different channels, considering cost per unit, audience reach, ROI, and spending limits. The objective is to maximize the total ROI across all channels, ensuring that spending does not exceed the budget and specific audience reach requirements are met. By integrating these various factors, our model seeks to provide a comprehensive solution to the complex problem of marketing budget allocation.

Potential challenges include acquiring accurate and comprehensive real-world data, capturing the complex relationships between marketing spend and outcomes within a linear framework, and accounting for the dynamic nature of marketing effectiveness across different channels. Addressing these challenges is crucial for the project's success. We will need to continually validate and refine our model to ensure its accuracy and applicability in real-world scenarios.

The rest of this report is structured as follows: Section 2 provides a detailed description of the data sources and their relevance to our model. Section 3 outlines the methodological framework, including the linear programming model and its components. Section 4 discusses the implementation of the model and the results obtained from initial simulations. Section 5 explores potential challenges and limitations of the model, and Section 6 concludes with a summary of findings and future directions for research.

**Data Collection and Preparation**: The dataset contains historical marketing data with variables such as campaign name, category, impressions, clicks, leads, revenue, and marketing spend. The data was loaded from a CSV file and preprocessed to calculate key metrics like ROI for each campaign.

## MATHEMATICAL MODEL
The optimization problem is formulated as a linear programming model using JuMP in Julia. The primary objective is to maximize the ROI, considering several constraints related to budget allocation and ensuring cost-efficiency in terms of cost per click (CPC) and cost per lead (CPL).


### 1. Parameters:

channels: Array of campaign names.
budget: Historical spending for each campaign.
roi: Return on investment, calculated as revenue / mark_spent.
clicks: Number of clicks for each campaign.
leads: Number of leads for each campaign.
max_spend: Maximum spend for each channel, set as 150% of historical spend.
total_budget: Total budget available for allocation.
min_total_roi: Minimum total ROI required.
cpc_threshold: Cost per click threshold.
cpl_threshold: Cost per lead threshold.
### 2. Decision Variables:

spend[i]: Amount of money allocated to channel i.

### 3. Objective Function:



### 4. Constraints:
**Total Budget Constraint**: The sum of spending across all channels should not exceed the total budget.

**Maximum Spend Constraint**: Ensure the spending on each channel does not exceed 150% of its historical spend.

**Minimum ROI Constraint**: Ensure the total ROI meets or exceeds the minimum required value.

**CPC Constraint**: Ensure the cost per click for each channel does not exceed the specified threshold.

**CPL Constraint**: Ensure the cost per lead for each channel does not exceed the specified threshold.


In [2]:
#Loading the data
using CSV
using DataFrames

file_path = "Marketing.csv"
data = CSV.read(file_path, DataFrame)

first(data, 5)

LoadError: ArgumentError: Symbol name may not contain \0

In [4]:
using CSV
using DataFrames
using JuMP
using GLPK

# Calculate ROI for each campaign
data[:, :ROI] = data.revenue ./ data.mark_spent

# Define the parameters
channels = data.campaign_name
budget = data.mark_spent
roi = data.ROI
clicks = data.clicks
leads = data.leads
max_spend = 1.5 .* budget  # Example maximum spend, can be adjusted
total_budget = 100000  # Example total budget constraint, you can adjust this
min_total_roi = 50000  # Example minimum ROI constraint, you can adjust this

# Defining thresholds for CPC and CPL (these should be set based on historical data or business standards)
cpc_threshold = 10.0  # Example CPC threshold, can be adjusted
cpl_threshold = 100.0  # Example CPL threshold, can be adjusted

model = Model(GLPK.Optimizer)

@variable(model, spend[i=1:length(channels)] >= 0, upper_bound = max_spend[i])

#objective function: Maximize ROI
@objective(model, Max, sum(roi[i] * spend[i] for i in 1:length(channels)))

#budget constraint
@constraint(model, sum(spend[i] for i in 1:length(channels)) <= total_budget)

# minimum ROI constraint
@constraint(model, sum(roi[i] * spend[i] for i in 1:length(channels)) >= min_total_roi)

# CPC constraints for each channel
for i in 1:length(channels)
    if clicks[i] > 0  # Avoid division by zero
        @constraint(model, spend[i] / clicks[i] <= cpc_threshold)
    end
end

# CPL constraints for each channel
for i in 1:length(channels)
    if leads[i] > 0  # Avoid division by zero
        @constraint(model, spend[i] / leads[i] <= cpl_threshold)
    end
end

optimize!(model)

optimal_spend = value.(spend)
total_roi = objective_value(model)

println("Optimal Spend per Channel:")
for i in 1:length(channels)
    println("Channel: ", channels[i], " Spend: ", optimal_spend[i])
end

println("Total ROI: ", total_roi)


Optimal Spend per Channel:
Channel: facebook_tier1 Spend: 0.0
Channel: facebOOK_tier2 Spend: 0.0
Channel: google_hot Spend: 0.0
Channel: google_wide Spend: 0.0
Channel: youtube_blogger Spend: 0.0
Channel: instagram_tier1 Spend: 0.0
Channel: instagram_tier2 Spend: 0.0
Channel: facebook_retargeting Spend: 0.0
Channel: facebook_lal Spend: 0.0
Channel: instagram_blogger Spend: 0.0
Channel: banner_partner Spend: 0.0
Channel: facebook_tier1 Spend: 0.0
Channel: facebOOK_tier2 Spend: 0.0
Channel: google_hot Spend: 0.0
Channel: google_wide Spend: 0.0
Channel: youtube_blogger Spend: 0.0
Channel: instagram_tier1 Spend: 0.0
Channel: instagram_tier2 Spend: 0.0
Channel: facebook_retargeting Spend: 0.0
Channel: facebook_lal Spend: 0.0
Channel: instagram_blogger Spend: 0.0
Channel: banner_partner Spend: 0.0
Channel: facebook_tier1 Spend: 0.0
Channel: facebOOK_tier2 Spend: 0.0
Channel: google_hot Spend: 0.0
Channel: google_wide Spend: 0.0
Channel: youtube_blogger Spend: 0.0
Channel: instagram_tier1 Spe

## Remaining Work

-Sensitivity Analysis- to understand how changes in input parameters (e.g., budget, CPC threshold) affect the outcomes.  
-Refining constraint parameters - based on insights from the sensitivity analysis. Adjusting the thresholds for CPC and CPL or setting different budget constraints. (Instead of the example value of 100,000, using the actual marketing budget available from the dataset,
replace the example value of (50,000) with a value derived from historical ROI data for minimum total ROI)  
-Visualization of results in the form of bar charts  
-Cleaning up results and comprehensive documentation

 Estimated time : 4 days

## Concerns
- Current model makes several assumptions, such as linear relationships between spending and ROI, and fixed CPC/CPL thresholds.Shall conduct scenario analysis to test the impact of different assumptions on the model's outcomes and adjust constraints accordingly
